# Chapter 11: Training Deep Neural Networks

Notebook ini merupakan hasil reproduksi dan penjelasan teori dari **Bab 11 - Training Deep Neural Networks** dari buku *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow (2nd Edition)* oleh Aurélien Géron.

📌 Bab ini membahas teknik-teknik penting dalam melatih Deep Neural Networks (DNN) secara efisien dan stabil, termasuk regularisasi, optimisasi, dan inisialisasi bobot.

---


## Ringkasan Teori Bab 11: Training Deep Neural Networks

### 1. Tantangan dalam Melatih Deep Neural Networks

- **Vanishing/Exploding Gradients**: Gradien terlalu kecil atau besar menyebabkan training gagal
- **Overfitting**: Model terlalu kompleks dan menyesuaikan noise dari data pelatihan
- **Training lambat atau tidak konvergen**

---

### 2. Strategi Inisialisasi Bobot

- **Glorot/Xavier Initialization**: ideal untuk sigmoid/tanh
- **He Initialization**: cocok untuk ReLU
```python
keras.layers.Dense(..., kernel_initializer="he_normal")
```

---

### 3. Normalisasi Fitur

Sebelum training, sangat penting untuk menormalisasi fitur agar distribusi input seragam.

---

### 4. Batch Normalization

Teknik ini menormalkan output dari layer sebelum fungsi aktivasi, mempercepat training dan membantu stabilitas.

```python
keras.layers.BatchNormalization()
```

---

### 5. Regularisasi

Menghindari overfitting:
- **Dropout**: menonaktifkan neuron secara acak saat training
```python
keras.layers.Dropout(0.5)
```
- **L1/L2 Regularization**: ditambahkan ke fungsi loss

---

### 6. Optimizers

Algoritma untuk memperbarui bobot:
- **SGD (Stochastic Gradient Descent)**
- **Momentum**
- **RMSProp**
- **Adam**: sangat populer dan efisien

```python
optimizer = keras.optimizers.Adam(learning_rate=0.001)
```

---

### 7. Early Stopping dan Callback

Menghentikan training jika validation loss berhenti membaik.

```python
keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
```

---

### 8. Learning Rate Scheduling

Menurunkan learning rate saat training berlangsung.

```python
keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
```

---

### 9. Simulasi Gradient Clipping

Membatasi besar gradien untuk menghindari exploding gradients.

```python
optimizer = keras.optimizers.Adam(clipnorm=1.0)
```

---


## Implementasi DNN dengan Regularisasi dan Callbacks

Berikut adalah implementasi Deep Neural Network (DNN) menggunakan teknik:
- **Batch Normalization**
- **Dropout**
- **Adam Optimizer dengan Gradient Clipping**
- **EarlyStopping & ReduceLROnPlateau callbacks**


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

# Load dan siapkan data
iris = load_iris()
X = iris.data
y = iris.target
y_cat = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Model DNN dengan regulasi
model = keras.models.Sequential([
    keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal", input_shape=X_train.shape[1:]),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(3, activation="softmax")
])

optimizer = keras.optimizers.Adam(learning_rate=0.01, clipnorm=1.0)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.summary()

In [ ]:
# Callback: EarlyStopping dan Learning Rate Scheduler
early_stop = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

# Training
history = model.fit(
    X_train, y_train, 
    epochs=100, 
    validation_split=0.2, 
    callbacks=[early_stop, reduce_lr],
    verbose=0
)

## Evaluasi Model dan Visualisasi

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Val Accuracy")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.tight_layout()
plt.show()

### Penjelasan:

- **Batch Normalization** mempercepat konvergensi dan menstabilkan training.
- **Dropout** mencegah overfitting dengan menonaktifkan neuron secara acak.
- **EarlyStopping** menghentikan training jika tidak ada perbaikan.
- **ReduceLROnPlateau** menurunkan learning rate otomatis saat stagnasi.
- **Gradient Clipping** menghindari exploding gradients dengan membatasi nilai gradien maksimum.

---
